# Gmail Toolkit in LangChain 

Mas informacion sobre el uso de la API de Gmail en [Guía de inicio rápido de Python | Gmail | Google for Developers](https://developers.google.com/gmail/api/quickstart/python?hl=es_419)

## Quickstart de la API:

In [ ]:
# Dependencies
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]


def main():
  """Shows basic usage of the Gmail API.
  Lists the user's Gmail labels.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)
    results = service.users().labels().list(userId="me").execute()
    labels = results.get("labels", [])

    if not labels:
      print("No labels found.")
      return
    print("Labels:")
    for label in labels:
      print(label["name"])

  except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

## Interaccion de la API con LangChain

Más informacion sobre este toolkit en: [Gmail Toolkit | 🦜️🔗 LangChain](https://python.langchain.com/docs/integrations/tools/gmail/)

**Pruebas con las herramientas (sin agente)**

In [ ]:
# Dependencies
%pip install langchain
%pip install -qU langchain-openai
%pip install -qU langchain-google-community[gmail]

In [2]:
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)
from langchain_google_community import GmailToolkit

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="../secrets/token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="../secrets/credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=69722697557-n13nrkrq7t630sthrt70fucdcmn19hsl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A49804%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=oqnuLKMd7TF2O1jXNwL1XV1hu0GWCq&access_type=offline


In [3]:
tools = toolkit.get_tools()
tools

[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x10d16cfe0>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x10d16cfe0>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x10d16cfe0>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x10d16cfe0>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x10d16cfe0>)]

In [4]:
import os
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model=os.getenv('OPENAI_MODEL'), api_key=os.getenv('OPENAI_API_KEY'))

In [5]:
model_with_tools = model.bind_tools(tools)

In [7]:
from langchain_core.messages import HumanMessage

response = model_with_tools.invoke([HumanMessage(content="Busca en mi correo el último mensaje de mi jefe, y respondele que lo he recibido.")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'search_gmail', 'args': {'query': 'from:tu_jefe@example.com', 'resource': 'messages', 'max_results': 1}, 'id': 'call_DwJOWOOAwZf66S0OI4SsaEb8', 'type': 'tool_call'}]


### Integracion de las herramientas con un Agente con LangGraph

Más información sobre construir un agente con LangGraph en: [Build an Agent | 🦜️🔗 LangChain](https://python.langchain.com/docs/tutorials/agents/#create-the-agent)

In [ ]:
# Dependencies
%pip install -qU langgraph

In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [10]:
# example_query = "Draft an email to fake@fake.com thanking them for coffee."
example_query = "Busca en mi correo el último mensaje de Amazon, y respondele que lo he recibido."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Busca en mi correo el último mensaje de Amazon, y respondele que lo he recibido.
================================== Ai Message ==================================
Tool Calls:
  search_gmail (call_OYpwabnl2E3RsRyLVbM1HiyS)
 Call ID: call_OYpwabnl2E3RsRyLVbM1HiyS
  Args:
    query: from:amazon
    resource: messages
    max_results: 1
================================= Tool Message =================================
Name: search_gmail

[{"id": "192a102712a3bc8c", "threadId": "192a102712a3bc8c", "snippet": "Hola Jorge, ¡Tu paquete ya se entregó! Rastrea tu paquete Devuelve o reemplaza los artículos en Mis Pedidos. Pedido # 702-2226614-5973852. Este correo electrónico se envió desde una dirección que no", "body": "www.amazon.com.mx\r\n--------------------------------------------------------------------------------------------------------------------------------------------\r\n\r\nHola Jorge,\r\n\r\n\r\n¡Tu paque

In [33]:
response = agent_executor.invoke(
    {"messages": [
        HumanMessage(content="Crea un borrador para fake@fake.com sobre ir a tomar un café."),
        ]}
)
response["messages"]

[HumanMessage(content='Crea un borrador para fake@fake.com sobre ir a tomar un café.', additional_kwargs={}, response_metadata={}, id='cafafb79-1339-4328-afb9-3f0513053168'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9Ue8EvVfbqYKYYALFl9rAJ6v', 'function': {'arguments': '{"message":"Hola, ¿te gustaría ir a tomar un café esta semana?","to":["fake@fake.com"],"subject":"Invitación para tomar un café"}', 'name': 'create_gmail_draft'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 575, 'total_tokens': 621, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-91490a0b-b098-4eaa-99e2-0bb4ec75dedd-0', tool_calls=[{'name': 'create_gmail_draft', 'args': {'message': 'Hola, ¿te gustaría ir a tomar un café esta semana?', 'to': ['fake@

### Con un propmt template

Más información sobre los prompt templates con Agentes en: [How to migrate from legacy LangChain agents to LangGraph | 🦜️🔗 LangChain](https://python.langchain.com/docs/how_to/migrate_agent/#prompt-templates)

**Para hacer uso del RAG y de la memoria, que van a ser input para el agente**

In [34]:
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Respond only in Spanish."),
        ("placeholder", "{messages}"),
    ]
)


def _modify_state_messages(state: AgentState):
    return prompt.invoke({"messages": state["messages"]}).to_messages() + [
        ("user", "Also say 'Pandamonium!' after the answer.")
    ]


langgraph_agent_executor = create_react_agent(
    model, tools, state_modifier=_modify_state_messages
)

query = "Draft an email to fake@fake.com thanking them for tea."

messages = langgraph_agent_executor.invoke({"messages": [("human", query)]})

print(
    {
        "input": query,
        "output": messages["messages"][-1].content,
    }
)

{'input': 'Draft an email to fake@fake.com thanking them for tea.', 'output': 'El correo ha sido redactado con el mensaje "¡Pandamonium!" al final. ¿Te gustaría enviarlo o hacer algún cambio?'}


In [11]:
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage

prompt = ChatPromptTemplate([
        ("system", """Eres un asistente útil. Responde solo en español.
         Junto con la pregunta te proporciono el contexto para que puedas responder, este 
         contexto lo vas a poder encontrar de la siguiente manera:
         Context: <contexto relacionado con la pregunta>
         A continuacion te presento la pregunta junto con el contexto:"""),
        ("placeholder", "{messages}"),
    ])


def _modify_state_messages(state: AgentState):
    return prompt.invoke({"messages": state["messages"]}).to_messages()


langgraph_agent_executor = create_react_agent(
    model, tools, state_modifier=_modify_state_messages
)

query = """Crea un borrador para JUAN para agradecerle por el café.
Context: JUAN es mi amigo de la secundaria y me invito a tomar un café en su casa, su correo es jorgeang03@outlook.com"""

messages = langgraph_agent_executor.invoke({
    "messages": [HumanMessage(content=query)]
    })

print(
    {
        "input": query,
        "output": messages["messages"][-1].content,
    }
)

{'input': 'Crea un borrador para JUAN para agradecerle por el café.\nContext: JUAN es mi amigo de la secundaria y me invito a tomar un café en su casa, su correo es jorgeang03@outlook.com', 'output': 'Se ha creado un borrador de correo para agradecer a JUAN por el café. Puedes revisarlo en tu bandeja de borradores.'}


In [49]:
query = """Crea un borrador para Beto para decirle lo que paso con la reunión.
Context: Beto es un companero del trabajo, y en la reunion se acordo que ya no sera parte de la empresa, su correo es jorgeang00@icloud.com"""

messages = langgraph_agent_executor.invoke({
    "messages": [HumanMessage(content=query)]
    })

print(
    {
        "input": query,
        "output": messages["messages"][-1].content,
    }
)

{'input': 'Crea un borrador para Beto para decirle lo que paso con la reunión.\nContext: Beto es un companero del trabajo, y en la reunion se acordo que ya no sera parte de la empresa, su correo es jorgeang00@icloud.com', 'output': 'Se ha creado un borrador del correo para informar a Beto sobre lo sucedido en la reunión. Puedes revisarlo en tus borradores con el ID: r-4153494255393875168.'}
